In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/rs6-attrition-predict/test.csv
/kaggle/input/rs6-attrition-predict/train.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import datetime

pd.set_option('display.max_columns', 50)

In [3]:
data = pd.read_csv('/kaggle/input/rs6-attrition-predict/train.csv')

In [4]:
data.head(20)

,user_id,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1374,58,No,Travel_Rarely,605,Sales,21,3,Life Sciences,1,1938,4,Female,72,3,4,Manager,4,Married,17875,11761,4,Y,Yes,13,3,3,80,1,29,2,2,1,0,0,0
1,1092,45,No,Travel_Rarely,950,Research & Development,28,3,Technical Degree,1,1546,4,Male,97,3,1,Research Scientist,4,Married,2132,4585,4,Y,No,20,4,4,80,1,8,3,3,5,4,0,3
2,768,40,No,Travel_Rarely,300,Sales,26,3,Marketing,1,1066,3,Male,74,3,2,Sales Executive,1,Married,8396,22217,1,Y,No,14,3,2,80,1,8,3,2,7,7,7,5
3,569,36,No,Non-Travel,1434,Sales,8,4,Life Sciences,1,789,1,Male,76,2,3,Sales Executive,1,Single,7587,14229,1,Y,No,15,3,2,80,0,10,1,3,10,7,0,9
4,911,25,Yes,Travel_Frequently,599,Sales,24,1,Life Sciences,1,1273,3,Male,73,1,1,Sales Representative,4,Single,1118,8040,1,Y,Yes,14,3,4,80,0,1,4,3,1,0,1,0
5,408,52,No,Travel_Rarely,1490,Research & Development,4,2,Life Sciences,1,546,4,Female,30,3,4,Manager,4,Married,16555,10310,2,Y,No,13,3,4,80,0,31,2,1,5,2,1,4
6,1321,47,No,Travel_Rarely,207,Research & Development,9,4,Life Sciences,1,1856,2,Female,64,3,1,Laboratory Technician,3,Single,2105,5411,4,Y,No,12,3,3,80,0,7,2,3,2,2,2,0
7,1224,26,No,Travel_Rarely,390,Research & Development,17,4,Medical,1,1718,4,Male,62,1,1,Laboratory Technician,3,Married,2305,6217,1,Y,No,15,3,3,80,3,3,3,4,3,2,0,2
8,1061,24,No,Non-Travel,830,Sales,13,2,Life Sciences,1,1495,4,Female,78,3,1,Sales Representative,2,Married,2033,7103,1,Y,No,13,3,3,80,1,1,2,3,1,0,0,0
9,530,27,No,Travel_Rarely,608,Research & Development,1,2,Life Sciences,1,725,3,Female,68,3,3,Manufacturing Director,1,Married,7412,6009,1,Y,No,11,3,4,80,0,9,3,3,9,7,0,7


In [5]:
def statics(data):
    stats = []
    for col in data.columns:
        stats.append((col, data[col].nunique(), data[col].isnull().sum() * 100 / data.shape[0],
                      data[col].value_counts(normalize=True, dropna=False).values[0] * 100, data[col].dtype))

    stats_df = pd.DataFrame(stats, columns=['Feature', 'Unique_values', 'Percentage_of_missing_values',
                                            'Percentage_of_values_in_the_biggest category', 'type'])
    stats_df.sort_values('Percentage_of_missing_values', ascending=False, inplace=True)
    return stats_df

statics(data)

,Feature,Unique_values,Percentage_of_missing_values,Percentage_of_values_in_the_biggest category,type
0,user_id,1176,0.0,0.085034,int64
1,Age,43,0.0,5.612245,int64
20,MonthlyRate,1147,0.0,0.170068,int64
21,NumCompaniesWorked,10,0.0,35.459184,int64
22,Over18,1,0.0,100.000000,object
23,OverTime,2,0.0,72.534014,object
24,PercentSalaryHike,15,0.0,14.540816,int64
25,PerformanceRating,2,0.0,84.948980,int64
26,RelationshipSatisfaction,4,0.0,31.037415,int64
27,StandardHours,1,0.0,100.000000,int64


In [6]:
data.describe(include='all')

,user_id,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1176.000000,1176.000000,1176,1176,1176.000000,1176,1176.000000,1176.000000,1176,1176.0,1176.000000,1176.000000,1176,1176.000000,1176.000000,1176.000000,1176,1176.000000,1176,1176.000000,1176.000000,1176.000000,1176,1176,1176.000000,1176.000000,1176.000000,1176.0,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.00000,1176.000000,1176.000000
unique,NaN,NaN,2,3,NaN,3,NaN,NaN,6,NaN,NaN,NaN,2,NaN,NaN,NaN,9,NaN,3,NaN,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,No,Travel_Rarely,NaN,Research & Development,NaN,NaN,Life Sciences,NaN,NaN,NaN,Male,NaN,NaN,NaN,Sales Executive,NaN,Married,NaN,NaN,NaN,Y,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,988,830,NaN,772,NaN,NaN,490,NaN,NaN,NaN,713,NaN,NaN,NaN,258,NaN,524,NaN,NaN,NaN,1176,853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,736.124150,36.805272,NaN,NaN,802.033163,NaN,9.159864,2.918367,NaN,1.0,1026.960034,2.750850,NaN,65.130102,2.724490,2.055272,NaN,2.732993,NaN,6458.690476,14247.159864,2.703231,NaN,NaN,15.152211,3.150510,2.714286,80.0,0.805272,11.161565,2.767007,2.764456,6.982143,4.19898,2.160714,4.098639
std,419.336964,9.065549,NaN,NaN,405.946729,NaN,8.137224,1.009809,NaN,0.0,594.763609,1.096221,NaN,20.294326,0.715027,1.106040,NaN,1.102477,NaN,4724.845883,7133.767499,2.521301,NaN,NaN,3.652543,0.357723,1.080583,0.0,0.865611,7.747576,1.250756,0.713251,6.094338,3.63124,3.208052,3.564190
min,0.000000,18.000000,NaN,NaN,104.000000,NaN,1.000000,1.000000,NaN,1.0,1.000000,1.000000,NaN,30.000000,1.000000,1.000000,NaN,1.000000,NaN,1009.000000,2094.000000,0.000000,NaN,NaN,11.000000,3.000000,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000
25%,374.750000,30.000000,NaN,NaN,463.500000,NaN,2.000000,2.000000,NaN,1.0,498.750000,2.000000,NaN,48.000000,2.000000,1.000000,NaN,2.000000,NaN,2858.750000,7912.750000,1.000000,NaN,NaN,12.000000,3.000000,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.00000,0.000000,2.000000
50%,742.500000,36.000000,NaN,NaN,805.500000,NaN,7.000000,3.000000,NaN,1.0,1031.000000,3.000000,NaN,65.000000,3.000000,2.000000,NaN,3.000000,NaN,4850.500000,14225.500000,2.000000,NaN,NaN,14.000000,3.000000,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.00000,1.000000,3.000000
75%,1101.250000,42.250000,NaN,NaN,1162.000000,NaN,14.000000,4.000000,NaN,1.0,1555.250000,4.000000,NaN,82.250000,3.000000,3.000000,NaN,4.000000,NaN,8380.250000,20372.500000,4.000000,NaN,NaN,18.000000,3.000000,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.00000,2.250000,7.000000


In [7]:
data['Attrition'] = np.where(data['Attrition']=='Yes',1,0)
data['Over18'] = np.where(data['Over18']=='Y',1,0)
data['OverTime'] = np.where(data['OverTime']=='Yes',1,0)

In [8]:
target = ['Attrition']
features = [i for i in data.columns if i not in ['Attrition','user_id','EmployeeNumber']]

In [9]:
data['Attrition'].value_counts()

0    988
1    188
Name: Attrition, dtype: int64

In [10]:
dvec=DictVectorizer(sparse=False)
train_features=dvec.fit_transform(data[features].to_dict(orient='record'))
print(dvec.feature_names_)
train_features.shape

['Age', 'BusinessTravel=Non-Travel', 'BusinessTravel=Travel_Frequently', 'BusinessTravel=Travel_Rarely', 'DailyRate', 'Department=Human Resources', 'Department=Research & Development', 'Department=Sales', 'DistanceFromHome', 'Education', 'EducationField=Human Resources', 'EducationField=Life Sciences', 'EducationField=Marketing', 'EducationField=Medical', 'EducationField=Other', 'EducationField=Technical Degree', 'EmployeeCount', 'EnvironmentSatisfaction', 'Gender=Female', 'Gender=Male', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole=Healthcare Representative', 'JobRole=Human Resources', 'JobRole=Laboratory Technician', 'JobRole=Manager', 'JobRole=Manufacturing Director', 'JobRole=Research Director', 'JobRole=Research Scientist', 'JobRole=Sales Executive', 'JobRole=Sales Representative', 'JobSatisfaction', 'MaritalStatus=Divorced', 'MaritalStatus=Married', 'MaritalStatus=Single', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'Over18', 'OverTime', 'PercentSalaryHike', 'Per

(1176, 53)

In [11]:
X_train,X_test,y_train,y_test = train_test_split(train_features,data[target].values,test_size=0.25,random_state=33)

In [12]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((882, 53), (294, 53), (882, 1), (294, 1))

In [13]:
ss = preprocessing.StandardScaler()
X_train_ss = ss.fit_transform(X_train)
X_test_ss = ss.transform(X_test)

# 测试集

In [14]:
test_data = pd.read_csv('/kaggle/input/rs6-attrition-predict/test.csv')

In [15]:
# test_data['Attrition'] = np.where(data['Attrition']=='Yes',1,0)
test_data['Over18'] = np.where(test_data['Over18']=='Y',1,0)
test_data['OverTime'] = np.where(test_data['OverTime']=='Yes',1,0)

In [16]:
test_features=dvec.transform(test_data[features].to_dict(orient='record'))
# print(dvec.feature_names_)
# test_features.shape

In [17]:
test = ss.transform(test_features)

# 逻辑回归

In [18]:
lr = LogisticRegression(solver='liblinear',multi_class='auto')
parameters = {"penalty":['l1','l2'],
             "C":np.logspace(-1, 1, 20)}
model = GridSearchCV(lr,param_grid=parameters,scoring='f1',n_jobs=-1, cv=5, verbose=True)
model.fit(X_train_ss,y_train)
print(model.best_params_)


Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:    2.6s


{'C': 1.8329807108324356, 'penalty': 'l2'}


[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    4.9s finished
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [19]:
# lr.fit(X_train_ss,y_train)
# y_predict = lr.predict(X_test_ss)
# print('LR准确率: %0.4lf' % accuracy_score(y_predict, y_test))

predictions = model.predict(X_test_ss)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.91      0.96      0.94       252
           1       0.67      0.43      0.52        42

    accuracy                           0.89       294
   macro avg       0.79      0.70      0.73       294
weighted avg       0.88      0.89      0.88       294



In [20]:
results = model.predict(test)

In [21]:
results2 = model.predict_proba(test)

In [22]:
pd.concat([test_data.user_id.to_frame(),pd.Series(results2[:,1]).to_frame().rename(columns={0:'Attrition'})],axis=1).to_csv('attrition_lr.csv')